<a href="https://colab.research.google.com/github/dunomi/dunomi/blob/master/projektNAstate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)


In [218]:
USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))

USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]


,Stan,Data,Przyrosty
33,Kansas,3/10/20,"[1.0, 1.0, 5.0, 1.6, 1.0, 1.38]"


In [0]:
USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100

  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))

In [0]:
USAdata=pd.merge(USAcases,USAweather)

In [0]:
np.cumprod(USAdata.Przyrosty[0])[5]
USAdata.head(5)

USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

# USAcumdata=USAcumdata[USAcumdata.Stan!='Louisiana']
USAcumdataLess50=USAcumdata[USAcumdata.Przyrosty<50]
USAcumdataMore50=USAcumdata[USAcumdata.Przyrosty>=50]

In [295]:
fig = px.scatter(x=USAcumdataLess50['Temperatura'].tolist(), 
                 y=USAcumdataLess50['Wilgotnosc'].tolist(),
                 size=USAcumdataLess50['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.update_yaxes(range=[0,1])
fig.show()

In [292]:
fig = px.scatter(x=USAcumdataMore50['Temperatura'].tolist(), 
                 y=USAcumdataMore50['Wilgotnosc'].tolist(),
                 size=USAcumdataMore50['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.show()

In [256]:
USAdata

,Stan,Data,Przyrosty,Temperatura,Wilgotnosc
0,Washington,3/10/20,"[1.37, 1.21, 1.29, 1.01, 1.12, 1.41]","[12, 12, 11, 3, 6, 8, 14]","[0.45, 0.45, 0.46, 0.91, 0.55, 0.38, 0.33]"
1,New York,3/10/20,"[1.27, 1.49, 1.28, 1.25, 1.39, 1.32]","[17, 8, 11, 13, 7, 8, 8]","[0.56, 0.36, 0.4, 0.63, 0.44, 0.27, 0.21]"
2,California,3/10/20,"[1.23, 1.25, 1.28, 1.21, 1.25, 1.31]","[19, 22, 23, 18, 14, 11, 13]","[0.45, 0.48, 0.38, 0.58, 0.71, 0.74, 0.63]"
3,Massachusetts,3/10/20,"[1.03, 1.14, 1.14, 1.12, 1.19, 1.2]","[19, 11, 6, 16, 12, 5, 7]","[0.62, 0.53, 0.77, 0.84, 0.32, 0.44, 0.42]"
4,Georgia,3/10/20,"[1.35, 1.35, 1.35, 1.57, 1.5, 1.22]","[18, 21, 24, 22, 27, 21, 14]","[0.76, 0.71, 0.69, 0.71, 0.49, 0.78, 0.78]"
5,Colorado,3/10/20,"[2.27, 1.32, 1.09, 2.06, 1.3, 1.22]","[16, 20, 8, 4, 10, 15, 16]","[0.27, 0.19, 0.55, 0.71, 0.56, 0.43, 0.35]"
6,Florida,3/10/20,"[1.87, 1.25, 1.43, 1.52, 1.51, 1.35]","[24, 25, 28, 27, 29, 29, 29]","[0.55, 0.57, 0.54, 0.56, 0.43, 0.4, 0.5]"
7,New Jersey,3/10/20,"[1.53, 1.26, 1.0, 2.38, 1.42, 1.82]","[18, 13, 11, 22, 13, 12, 7]","[0.6, 0.36, 0.72, 0.57, 0.27, 0.33, 0.55]"
8,Oregon,3/10/20,"[1.27, 1.26, 1.25, 1.07, 1.12, 1.08]","[17, 14, 12, 4, 7, 10, 16]","[0.44, 0.6, 0.49, 0.86, 0.79, 0.43, 0.54]"
9,Texas,3/10/20,"[1.62, 1.29, 1.59, 1.33, 1.26, 1.18]","[27, 29, 30, 28, 29, 17, 23]","[0.61, 0.52, 0.49, 0.55, 0.56, 0.87, 0.74]"
